In [1]:
#Imports
import pandas as pd
import numpy as np
import os
import glob

In [2]:
def y(df):

    return df[
        (df['DIA_NAC'] > 0) & (df['DIA_NAC'] <= 31) &
        (df['PESO'] > 0) & (df['PESO'] < 9999) & 
        (df['TALLA'] > 0) & (df['TALLA'] < 99) & 
        (df['EDAD_P'] > 0) & (df['EDAD_P'] < 99) &
        (df['EDAD_M'] > 0) & (df['EDAD_M'] < 99)
    ]

In [3]:
#1.a
def loadDataFramesFromCSV(folder, indexKey) :
        
    files = [os.path.basename(x) for x in glob.glob(folder)]
    
    mapData = []

    for fileName in files:
        mapData.append(pd.read_csv(folder[0: -5] + fileName, sep=';', index_col=None, header=0, error_bad_lines=False))
       
    return pd.concat(mapData)

nacimientos = y(loadDataFramesFromCSV(os.getcwd() + "/datasets/nacimientos/*.csv", 4))
nacimientos.describe()

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3357: DtypeWarning: Columns (21) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


,SEXO,DIA_NAC,MES_NAC,ANO_NAC,TIPO_PARTO,ATENC_PART,LOCAL_PART,SEMANAS,PESO,TALLA,...,ACTIV_M,CATEG_M,COMUNA,URBA_RURAL,HIJ_VIVOS,HIJ_FALL,HIJ_MORT,HIJ_TOTAL,REG_RES,SERV_RES
count,4.357145e+06,4.357145e+06,4.357145e+06,4.357145e+06,4.357145e+06,4.357145e+06,4.357145e+06,4.357145e+06,4.357145e+06,4.357145e+06,...,4.357145e+06,4.357036e+06,4.357145e+06,4.357145e+06,4.357145e+06,4.357145e+06,4.357145e+06,4.357145e+06,4.349890e+06,4.349890e+06
mean,1.489368e+00,1.570983e+01,6.507575e+00,2.005238e+03,1.020268e+00,1.607075e+00,1.007979e+00,3.863606e+01,3.331992e+03,4.935554e+01,...,3.505637e-01,8.072685e-01,6.841074e+03,1.106166e+00,2.183714e+00,2.575269e-01,2.573199e-01,2.029748e+00,9.236104e+00,1.308301e+01
std,5.009657e-01,8.774170e+00,3.460860e+00,6.434131e+00,1.652056e-01,4.947723e-01,1.359138e-01,2.047545e+00,5.428390e+02,2.605444e+00,...,6.304813e-01,1.221793e+00,5.214765e+03,3.107719e-01,4.834507e+00,4.893503e+00,4.893904e+00,2.336332e+00,3.836555e+00,6.344831e+00
min,1.000000e+00,1.000000e+00,1.000000e+00,1.996000e+03,1.000000e+00,1.000000e+00,1.000000e+00,1.500000e+01,9.000000e+01,9.000000e+00,...,0.000000e+00,0.000000e+00,2.010000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00
25%,1.000000e+00,8.000000e+00,3.000000e+00,2.000000e+03,1.000000e+00,1.000000e+00,1.000000e+00,3.800000e+01,3.050000e+03,4.800000e+01,...,0.000000e+00,0.000000e+00,6.310000e+02,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,6.000000e+00,9.000000e+00
50%,1.000000e+00,1.600000e+01,7.000000e+00,2.004000e+03,1.000000e+00,2.000000e+00,1.000000e+00,3.900000e+01,3.360000e+03,5.000000e+01,...,0.000000e+00,0.000000e+00,7.109000e+03,1.000000e+00,2.000000e+00,0.000000e+00,0.000000e+00,2.000000e+00,1.000000e+01,1.300000e+01
75%,2.000000e+00,2.300000e+01,1.000000e+01,2.009000e+03,1.000000e+00,2.000000e+00,1.000000e+00,4.000000e+01,3.670000e+03,5.100000e+01,...,1.000000e+00,2.000000e+00,1.311000e+04,1.000000e+00,3.000000e+00,0.000000e+00,0.000000e+00,3.000000e+00,1.300000e+01,1.700000e+01
max,9.000000e+00,3.100000e+01,1.200000e+01,2.017000e+03,9.000000e+00,9.000000e+00,9.000000e+00,9.900000e+01,6.595000e+03,6.400000e+01,...,9.000000e+00,9.000000e+00,1.520200e+04,9.000000e+00,9.900000e+01,9.900000e+01,9.900000e+01,9.900000e+01,1.500000e+01,3.300000e+01


In [4]:
#1.b
nacimientos['MES_NAC'].mode()
#El mes de naciemiento mas comun, es el mes de Enero, dentro de todos los dataframes, asumiendo 
#los predicados propuestos en consulta general

0    1
dtype: int64

In [5]:
#1.c
nacimientos['DIA_NAC'].mode()
#El dia de nacimiento mas comun, es el dia 5, asumiendo los predicados propuestos en consulta general

0    5
dtype: int64

In [6]:
covar_general = nacimientos.cov()

In [7]:
#1.d.I
covar_general['PESO']['TALLA']

#La covarianza entre peso y talla de todo el dataframe es 1151.57, esto quiere decir que hay fuerte 
#relacion entre las dos variables

1151.5787831593882

In [8]:
def covar(var1, var2) :

    dataframe_groupped = nacimientos.groupby(by = ['ANO_NAC']).agg(
        d_max=pd.NamedAgg(column="ANO_NAC", aggfunc="max")
    )
     
    anios = dataframe_groupped.reset_index()['ANO_NAC']
    
    for anio in anios:
        
        covar_vars = nacimientos[
            nacimientos['ANO_NAC'] == anio
        ].cov()[var1][var2]
        
        print("año:" , anio , "covarianza", var1, var2, covar_vars)

In [9]:
def correl(var1, var2) :

    dataframe_groupped = nacimientos.groupby(by = ['ANO_NAC']).agg(
        d_max=pd.NamedAgg(column="ANO_NAC", aggfunc="max")
    )
     
    anios = dataframe_groupped.reset_index()['ANO_NAC']
    
    for anio in anios:
        
        correl_vars = nacimientos[
            nacimientos['ANO_NAC'] == anio
        ].corr()[var1][var2]
        
        print("año:" , anio , "correlacion", var1, var2, correl_vars)

In [10]:
#1.d.I
covar('PESO', 'TALLA')

#Se observa que la covarianza, entre todos los años, se mantiene fuerte entre las variables peso-talla, 
#es decir la relacion entre estas variables es fuerte y se mantiene uniforme

año: 1996 covarianza PESO TALLA 1085.4653360823056
año: 1997 covarianza PESO TALLA 1088.3400018104194
año: 1998 covarianza PESO TALLA 1124.13443444404
año: 1999 covarianza PESO TALLA 1137.0183582781808
año: 2000 covarianza PESO TALLA 1138.420171211992
año: 2001 covarianza PESO TALLA 1147.7090440657348
año: 2002 covarianza PESO TALLA 1169.0082515832257
año: 2003 covarianza PESO TALLA 1198.7999655229933
año: 2004 covarianza PESO TALLA 1188.5271018882004
año: 2005 covarianza PESO TALLA 1160.311739665922
año: 2006 covarianza PESO TALLA 1201.6511336237368
año: 2007 covarianza PESO TALLA 1175.6471654581483
año: 2008 covarianza PESO TALLA 1120.9805930010443
año: 2009 covarianza PESO TALLA 1073.4128359626104
año: 2012 covarianza PESO TALLA 1113.9488471890493
año: 2013 covarianza PESO TALLA 1122.32420142102
año: 2015 covarianza PESO TALLA 1222.3854692961634
año: 2016 covarianza PESO TALLA 1206.5779894345624
año: 2017 covarianza PESO TALLA 1206.4964545510766


In [11]:
#1.d.II
correl('PESO', 'TALLA')

#Se observa que la correlacion entre las variables y entre los años de estudio, se mantiene como asociacion alta

año: 1996 correlacion PESO TALLA 0.8069518187611212
año: 1997 correlacion PESO TALLA 0.8089060125742329
año: 1998 correlacion PESO TALLA 0.8182298157917199
año: 1999 correlacion PESO TALLA 0.8177874503114567
año: 2000 correlacion PESO TALLA 0.8139722864561552
año: 2001 correlacion PESO TALLA 0.811966261784761
año: 2002 correlacion PESO TALLA 0.8091638292938252
año: 2003 correlacion PESO TALLA 0.8094935548666183
año: 2004 correlacion PESO TALLA 0.806082773838522
año: 2005 correlacion PESO TALLA 0.8048940634176609
año: 2006 correlacion PESO TALLA 0.8096761210853728
año: 2007 correlacion PESO TALLA 0.8067258011262431
año: 2008 correlacion PESO TALLA 0.8219082360418786
año: 2009 correlacion PESO TALLA 0.8240821791922216
año: 2012 correlacion PESO TALLA 0.8417518005847248
año: 2013 correlacion PESO TALLA 0.8404412586738272
año: 2015 correlacion PESO TALLA 0.816631793824781
año: 2016 correlacion PESO TALLA 0.7976631179418455
año: 2017 correlacion PESO TALLA 0.8307449477043788


In [12]:
covar_general['EDAD_P']['EDAD_M']

#La covarianza entre la edad del padre y edad de la madre de todo el dataframe es 27.7890, esto quiere 
#decir que hay fuerte relacion entre las dos variables de estudio

37.11396757811241

In [13]:
#1.e.I
covar('EDAD_P', 'EDAD_M')

#Se observa que la covarianza, entre todos los años, se mantiene fuerte entre las variables 
#edad del padre y edad de la madre, es deci la relacion entre estas variables es fuerte, pero en algunos casos 
#cae fuerte esta relacion, por ejemplo en los años 2004 y 2006 (puede deberse a que hay algunas personas, en la 
#columa de la edad del padre, marcadas con 99 años)

año: 1996 covarianza EDAD_P EDAD_M 33.15393468330001
año: 1997 covarianza EDAD_P EDAD_M 33.89638533156794
año: 1998 covarianza EDAD_P EDAD_M 34.67893443501924
año: 1999 covarianza EDAD_P EDAD_M 35.21948537746692
año: 2000 covarianza EDAD_P EDAD_M 36.02854532349857
año: 2001 covarianza EDAD_P EDAD_M 37.01806604993652
año: 2002 covarianza EDAD_P EDAD_M 37.3353295798695
año: 2003 covarianza EDAD_P EDAD_M 37.2946311510757
año: 2004 covarianza EDAD_P EDAD_M 36.76614155550892
año: 2005 covarianza EDAD_P EDAD_M 38.77758601793898
año: 2006 covarianza EDAD_P EDAD_M 39.14051584360457
año: 2007 covarianza EDAD_P EDAD_M 38.957716267068214
año: 2008 covarianza EDAD_P EDAD_M 39.36094388341994
año: 2009 covarianza EDAD_P EDAD_M 39.4874669766799
año: 2012 covarianza EDAD_P EDAD_M 39.52664728517217
año: 2013 covarianza EDAD_P EDAD_M 38.78129406704523
año: 2015 covarianza EDAD_P EDAD_M 36.610258386036236
año: 2016 covarianza EDAD_P EDAD_M 35.280384108889024
año: 2017 covarianza EDAD_P EDAD_M 33.92452997

In [14]:
#1.e.II
correl('EDAD_P', 'EDAD_M')

#Se observa que la correlacion entre las variables de la edad del padre y de la madre, tienen alta relacion, 
#respecto a cada año de estudio. En el año 2004 y 2006 y tal como se puede apreciar en la covarianza, si bien 
#se tiene una correlacion positiva, esta es baja, y puede deberse a los mismos factores descritos en el enunciado
#de la covarianza

año: 1996 correlacion EDAD_P EDAD_M 0.7191033622249087
año: 1997 correlacion EDAD_P EDAD_M 0.7247403683862226
año: 1998 correlacion EDAD_P EDAD_M 0.7296278053762393
año: 1999 correlacion EDAD_P EDAD_M 0.72940470423727
año: 2000 correlacion EDAD_P EDAD_M 0.7333586527520604
año: 2001 correlacion EDAD_P EDAD_M 0.737944562598619
año: 2002 correlacion EDAD_P EDAD_M 0.7385779153676287
año: 2003 correlacion EDAD_P EDAD_M 0.736209782351795
año: 2004 correlacion EDAD_P EDAD_M 0.7159289710161831
año: 2005 correlacion EDAD_P EDAD_M 0.7440481912650658
año: 2006 correlacion EDAD_P EDAD_M 0.7307681055553491
año: 2007 correlacion EDAD_P EDAD_M 0.7322004012098318
año: 2008 correlacion EDAD_P EDAD_M 0.7366510318168198
año: 2009 correlacion EDAD_P EDAD_M 0.7394125909248829
año: 2012 correlacion EDAD_P EDAD_M 0.7480015240640328
año: 2013 correlacion EDAD_P EDAD_M 0.7455483836770453
año: 2015 correlacion EDAD_P EDAD_M 0.7375315938224521
año: 2016 correlacion EDAD_P EDAD_M 0.7303860541985884
año: 2017 corr